In [1]:
""" GanttProject csv file generator 

Helper script for generating CSV files for proper importing into GanttProject """

' GanttProject csv file generator \n\nHelper script for generating CSV files for proper importing into GanttProject '

In [2]:
_fields_str = "ID,Name,Begin date,End date,Duration,Completion,Cost,Coordinator,Predecessors,Outline number,Resources,Web Link,Notes"
_fields = _fields_str.split(",")
_fields2 = [(f, f.lower().replace(" ", "_")) for f in _fields]

In [32]:
import itertools
import io
import datetime
import collections

_id_counter = itertools.count(1)
def _get_id():
    global _id_counter
    return next(_id_counter)


class _GanttData():
    """ POD class for CSV data representation. """
    _fields_ = _fields2
    def __init__(self, id=0, name="",begin_date="",
                     end_date="",duration="",completion="",
                     cost="",coordinator="",predecessors="",
                     outline_number="",resources="",web_link="",notes=""):
        
        # fields corresponding to csv fields
        self.id = id
        self.name = name
        self.begin_date = begin_date
        self.end_date = end_date
        self.duration = duration
        self.completion = completion
        self.cost = cost
        self.coordinator = coordinator
        self.predecessors = predecessors
        self.outline_number = outline_number
        self.resources = resources
        self.web_link = web_link
        self.notes = notes
        
    def csv_line(self):
        return ",".join(str(getattr(self, attr)) for _, attr in self._fields_)
    
    

class GanttTask(_GanttData):
    _fields_ = _fields2
    def __init__(self, name, project, parent=None, id=None, begin_date=None,
                         duration=1, completion=0,
                         cost="",coordinator="",predecessors="", resources="",web_link="",notes=""):
        
        super().__init__(id, name, begin_date, "", duration, completion,
                               cost, coordinator, predecessors, "", resources, web_link, notes)
        
        # internal use
        self.project = project    # project reference
        self._subtasks = []       # child tasks
        self.parent = parent       # parent task
    
    def write(self, b):
        b.write(self.csv_line())
        for c in self._subtasks:
            c.write(b)
    
    def add_subtask(self, c):
        self._subtasks.append(c)
        
    def subtask(self, name="",begin_date="",
                         end_date="",duration="",completion="",
                         cost="",coordinator="",predecessors="",
                         resources="",web_link="",notes=""):
        t = self.parent.subtask(self, name, begin_date, duration, completion, cost, coordinator, 
                                predecessors, resources, web_link, notes)
        return t
        
    def count_subtasks(self):
        n = 0
        n += len(self._subtasks)
        for c in self._subtasks:
            n += c.count_subtasks()
        return n
    
    def iter_tasks(self):
        for c in self._subtasks:
            yield c
            yield from c._subtasks
            
    def index(self, item):
        return self._subtasks.index(item)
    
    def resolve_outlines(self, ol):
        self.outline_number = ol
        for i, st in enumerate(self._subtasks, 1):
            s = str(i)
            st.resolve_outlines(ol + "." + s)
        
class GanttProject():
    _fields_ = _fields2
    def __init__(self, name="Project"):
        self.name = name
        self._tasks = []
        self._id_counter = itertools.count(1)
        
    def __getitem__(self, i):
        return self._tasks[i]
        
    def _next_id(self):
        #n = 0
        #n += len(self._tasks)
        #for t in self._tasks:
        #    n += t.count_subtasks()
        #return n
        #return next(self._id_counter)
        return self.total_tasks()
    
    def total_tasks(self):
        n = len(self._tasks)
        for t in self._tasks:
            n += t.count_subtasks()
        return n
        
    def new_task(self, name="", begin_date=None,
                         duration=None, completion=0,
                         cost=0, coordinator="", predecessors="", 
                         resources="", web_link="", notes=""):
        id = self._next_id()
        t = GanttTask(name, self, self, id, begin_date, duration, completion, cost, coordinator, 
                     predecessors, resources, web_link, notes)
        self._tasks.append(t)
        t.outline_number = self._calc_outline(self, t)
        return t
    
    def _calc_outline(self, parent, task):
        if self is parent:
            return str(self._tasks.index(task)+1)
        else:
            return parent.outline_number + "." + str(parent.index(task) + 1)
    
    def subtask(self, parent, name="", begin_date=None,
                         duration=None, completion=0,
                         cost=0, coordinator="", predecessors="", 
                        resources="", web_link="", notes=""):
        id = self._next_id()
        t = GanttTask(name, self, parent, id, begin_date, duration, completion, cost, 
                      coordinator, predecessors, resources, web_link, notes)
        parent.add_subtask(t)
        t.outline_number = self._calc_outline(parent, t)
        return t
    
    def iter_tasks(self):
        for t in self._tasks:
            yield t
            yield from t.iter_tasks()
        
    def write(self, b):
        b.write(",".join(f for f, _ in self._fields_))
        b.write("\n")
        for t in self.iter_tasks():
            if not t.outline_number:
                t.outline_number = t.id
            if not t.duration:
                t.duration = 1
            if not t.begin_date:
                t.begin_date = _today()
            b.write(t.csv_line())
            b.write("\n")
    
    def to_csv(self):
        b = io.StringIO()
        self.write(b)
        return b.getvalue()
    
    
def _fmt_date(d):
    return d.strftime("%m/%d/%y")
    
def _today():
    return _fmt_date(_today_dt())

def _today_dt():
    return datetime.datetime.now()

def _today_plus(days=1):
    return _fmt_date(_today_dt() + datetime.timedelta(days=days))

In [33]:
tasks = """Updated Deliverables
 Purpose Statement
 User Requirements Specification
 Risk Analysis
 Project Plan
 Product Requirement Checklist
 Functional Requirements Specification
 Software Design Specification
LabVIEW Code Review
Commit Log
Requirements and Code Matrix
Requirements and Unit Test Matrix
Unit Testing Report
Phase 2 Completion Approval""".splitlines()
p = GanttProject()
for s in tasks:
    if s.startswith(" "):
        p[-1].subtask(s)
    else:
        t = p.new_task(s)
s=p.to_csv()
print(s)

ID,Name,Begin date,End date,Duration,Completion,Cost,Coordinator,Predecessors,Outline number,Resources,Web Link,Notes
0,Updated Deliverables,06/21/16,,1,0,0,,,1,,,
1, Purpose Statement,06/21/16,,1,,,,,1.1,,,
2, User Requirements Specification,06/21/16,,1,,,,,1.2,,,
3, Risk Analysis,06/21/16,,1,,,,,1.3,,,
4, Project Plan,06/21/16,,1,,,,,1.4,,,
5, Product Requirement Checklist,06/21/16,,1,,,,,1.5,,,
6, Functional Requirements Specification,06/21/16,,1,,,,,1.6,,,
7, Software Design Specification,06/21/16,,1,,,,,1.7,,,
8,LabVIEW Code Review,06/21/16,,1,0,0,,,2,,,
9,Commit Log,06/21/16,,1,0,0,,,3,,,
10,Requirements and Code Matrix,06/21/16,,1,0,0,,,4,,,
11,Requirements and Unit Test Matrix,06/21/16,,1,0,0,,,5,,,
12,Unit Testing Report,06/21/16,,1,0,0,,,6,,,
13,Phase 2 Completion Approval,06/21/16,,1,0,0,,,7,,,



In [14]:
with open("C:/.replcache/gantt_test.csv", 'w') as f:
    f.write(p.to_csv())

In [15]:
p[1].name

'LabVIEW Code Review'